In [1]:
from ecnet.datasets import load_cn, load_ysi, load_kv, load_cp, load_lhv

smiles_cn, cn = load_cn()
smiles_ysi, ysi = load_ysi()
smiles_kv, kv = load_kv()
smiles_cp, cp = load_cp()
smiles_lhv, lhv = load_lhv()

print(len(cn), len(ysi), len(kv), len(cp), len(lhv))

460 558 213 43 388


In [2]:
from ecnet.datasets import QSPRDataset

ds_cn = QSPRDataset(smiles_cn, cn, backend='alvadesc')
ds_ysi = QSPRDataset(smiles_ysi, ysi, backend='alvadesc')
ds_kv = QSPRDataset(smiles_kv, kv, backend='alvadesc')
ds_cp = QSPRDataset(smiles_cp, cp, backend='alvadesc')
ds_lhv = QSPRDataset(smiles_lhv, lhv, backend='alvadesc')

In [3]:
from ecnet.tasks import select_rfr

idx_cn, imp_cn = select_rfr(ds_cn, n_estimators=100)
idx_ysi, imp_ysi = select_rfr(ds_ysi, n_estimators=100)
idx_kv, imp_kv = select_rfr(ds_kv, n_estimators=100)
idx_cp, imp_cp = select_rfr(ds_cp, n_estimators=100)
idx_lhv, imp_lhv = select_rfr(ds_lhv, n_estimators=100)

In [4]:
names_cn = [ds_cn.desc_names[i] for i in idx_cn]
names_ysi = [ds_ysi.desc_names[i] for i in idx_ysi]
names_kv = [ds_kv.desc_names[i] for i in idx_kv]
names_cp = [ds_cp.desc_names[i] for i in idx_cp]
names_lhv = [ds_lhv.desc_names[i] for i in idx_lhv]

In [5]:
from csv import DictWriter

headers = ['Property', 'Desc. Rank', 'Desc. Imp.', 'Desc. Name']

rows_cn = []
rows_ysi = []
rows_kv = []
rows_cp = []
rows_lhv = []    
for i in range(10):
    rows_cn.append({
        'Property': 'CN',
        'Desc. Rank': i + 1,
        'Desc. Imp.': imp_cn[i],
        'Desc. Name': names_cn[i]
    })
    rows_ysi.append({
        'Property': 'YSI',
        'Desc. Rank': i + 1,
        'Desc. Imp.': imp_ysi[i],
        'Desc. Name': names_ysi[i]
    })
    rows_kv.append({
        'Property': 'KV',
        'Desc. Rank': i + 1,
        'Desc. Imp.': imp_kv[i],
        'Desc. Name': names_kv[i]
    })
    rows_cp.append({
        'Property': 'CP',
        'Desc. Rank': i + 1,
        'Desc. Imp.': imp_cp[i],
        'Desc. Name': names_cp[i]
    })
    rows_lhv.append({
        'Property': 'LHV',
        'Desc. Rank': i + 1,
        'Desc. Imp.': imp_lhv[i],
        'Desc. Name': names_lhv[i]
    })

with open('results/descriptor_ranks.csv', 'w', encoding='utf8') as csv_file:
    writer = DictWriter(csv_file, headers, delimiter=',', lineterminator='\n')
    writer.writeheader()
    writer.writerows(rows_cn)
    writer.writerows(rows_ysi)
    writer.writerows(rows_kv)
    writer.writerows(rows_cp)
    writer.writerows(rows_lhv)